# Note book imports

In [8]:
from os.path import exists, join
from os import walk
import pandas as pd

# Constants

In [22]:
EXAMPLE_FILE = "../spamData/01_Processing/practice_email.txt"
EASY_NONSPAM_1 = "../SpamData/01_Processing/spam_assassin_corpus/easy_ham_1/"
EASY_NONSPAM_2 = "../SpamData/01_Processing/spam_assassin_corpus/easy_ham_2/"
SPAM_1 = "../SpamData/01_Processing/spam_assassin_corpus/spam_1/"
SPAM_2 = "../SpamData/01_Processing/spam_assassin_corpus/spam_2/"

SPAM_CAT = 1
HAM_CAT = 0

In [3]:
with open(EXAMPLE_FILE, "r", encoding="latin-1") as ex_in:
    is_body = False
    lines = []

    for line in ex_in.readlines():
        if is_body:
            lines.append(line)
        elif line == "\n":
            is_body = True
email_body = "".join(lines)
print(email_body)


Dear Mr Still

Good tidings to you and all your staff for the festive season ahead (Christmas).
Now to the crux of the matter-in-hand: I am a fully qualified Santa Claus and am wondering whether you might consider me to run my own "Santa's Grotto" in your store.
But WAIT! You're probably thinking: "What makes him so special?"
Well, first of all, I have made several changes to the characterisation of Father Christmas. Rather than greeting the children with shouts of "Ho, ho, ho!" I prefer to whisper the phrase "Dependence is not unfathomable in this cruel world we live in". In addition, my gifts are ALL hand-made, ranging from felt hoops to vanilla-pod holders.
You will note also, from the enclosed sketch, that I have radically redesigned Santa's outfit and have renamed my character "Lord Buckles". Would you be interested in employing me? I promise NEVER to let you down.
I look forward to hearing from you.

Best wishes
Robin Cooper
[Excerpt from the book: The Timewaster Letters by Robi

## Generator function

In [15]:
def email_body_generator(rel_path: str):
    """"
    Walks a directory and yields each file from the directory
    :param rel_path: A relative path to a folder.
    :yield : A tuple of two strings, the file name and a body of an email
    """
    for root, dirnames, filenames in walk(rel_path):
        for file_name in filenames:
            file_path = join(root, file_name)
            with open(file_path, "r", encoding="latin-1") as ex_in:
                is_body = False
                lines = []

                for line in ex_in.readlines():
                    if is_body:
                        lines.append(line)
                    elif line == "\n":
                        is_body = True
            email_body = "".join(lines)
            yield file_name, email_body


In [16]:
def df_from_directory(rel_path: str, classification):
    """
    Loops through a generator function reading email body and names
    """
    rows = []
    row_names = []
    for file_name, email_body in email_body_generator(rel_path):
        rows.append({"MESSAGE": email_body, "CATEGORY": classification})
        row_names.append(file_name)
    return pd.DataFrame(rows, index=row_names)

In [25]:
spam_emails = df_from_directory(SPAM_1, SPAM_CAT)
spam_emails = spam_emails.append(df_from_directory(SPAM_2, SPAM_CAT))

ham_emails = df_from_directory(EASY_NONSPAM_1, HAM_CAT)
ham_emails = ham_emails.append(df_from_directory(EASY_NONSPAM_2, HAM_CAT))

C:\Users\scott\AppData\Local\Temp\ipykernel_8984\1403818923.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  spam_emails = spam_emails.append(df_from_directory(SPAM_2, SPAM_CAT))
C:\Users\scott\AppData\Local\Temp\ipykernel_8984\1403818923.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ham_emails = ham_emails.append(df_from_directory(EASY_NONSPAM_2, HAM_CAT))


(3901, 2)

In [27]:
data = pd.concat([spam_emails, ham_emails])
print(f"Shape of entire df {data.shape}")

Shape of entire df (5799, 2)


,MESSAGE,CATEGORY
01396.61983fbe6ec43f55fd44e30fce24ffa6,http://news.bbc.co.uk/1/hi/england/2515127.stm...,0
01397.9f9ef4c2a8dc012d80f2ce2d3473d3b7,"> >-- be careful when using this one.) Also, t...",0
01398.169b51731fe569f42169ae8f948ec676,">>>>> ""SM"" == Skip Montanaro <skip@pobox.com> ...",0
01399.ca6b00b7b341bbde9a9ea3dd6a7bf896,"So then, ""Mark Hammond"" <mhammond@skippinet.co...",0
01400.f897f0931e461e7b2e964d28e927c35e,"Hi there,\n\nNow this is probably of no use to...",0
